Anyone who is managing a business/product knows the importance of a business intelligence dashboard. There are a host of proprietory paid tools like Tableau, Qlikview etc which dominate this segment. But these are typically too expensive and an overkill for small startups/teams. So we will explore some simple, free to use, open source alternatives that one can use to obtain business intelligence.

### **Setting up the data source**

The data about our site/app is available in various places - as Google analytics data, as Mixpanel metrics etc. But let's first focus only on the core business data stored in our own database.

The first step is to set up a datasource from which we can query our business data. We should **never run analytics queries directly on the production database** - as these queries typically tend to be long running - and will adversely interfere with our application's transactional queries. We should instead set up a separate analytics database which will collect data from the production db.


#### **Redshift**
The preferred approach is to set up a data warehouse using a product specifically designed for running analytical queries. **Amazon Redshift** is the most recommended solution for a data warehouse since it is specifically designed to be able to handle large analytical queries whereas the traditional RDBMS systems like Postgres & MySQL are optimized for shorter, but more frequent query loads.

There are some good tutorials available online for setting up a redshift cluster, like this one for example - https://blog.panoply.io/how-to-move-your-mysql-to-amazon-redshift. But our work is not done with the initial setup. We also need to have a way of continuously syncing the data warehouse updated with our production database. Writing cron scripts to sync only the modified data is not a trivial task at all. AWS RDS has integrations with some tools like Flywheel, Attunity which do this for us. But these are also quite expensive.

Redshift is best suited for those cases where the data volume is too large and a standard relational db proves to be inefficient in executing the queries on time. Considering the problems listed above, it will be better for us to come up with a simpler solution

#### **Read Replica - poor man's data warehouse**

A very simple solution is to just set up a read replica of the database. This is very easy if you are using AWS RDS. The database instance will have an Actions menu which has an option - "Create Read Replica". The replica can be created by filling the form shown next. For the replica, one can choose a lower priced instance than the one used for the production database. (For a detailed guide refer - https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/USER_ReadRepl.html#USER_ReadRepl.Create )

If you are not using RDS for the database, you can still find resources online to find out how to set up a master-slave replication for your database. For MySQL there is a tutorial available here - https://www.digitalocean.com/community/tutorials/how-to-set-up-master-slave-replication-in-mysql

So, with this, we have a data source ready for querying. Lets summarize the advantages of this approach once
    1. Very easy to set up and quite inexpensive ( In RDS, if you use a smaller instance for the replica, it will cost only a small fraction of the amount you pay for your production db )
    2. Syncing the data with production db is automatically taken care of
    3. No separate schema needs to be created for the data warehouse. As you grow in size, this can start becoming a disadvantage. But since we are discussing the needs of early stage startups, having the same schema for your production and analytics databases is definitely an advantage since you are short on manpower and time. Your backend engineers already know your production schema very well. They can easily pitch in to write your analytics queries as well
    4. Since it is a read replica, there is no danger of the data being modified accidentally by some query.

#### **Data Querying and Visualization - Available Options**

